# LangChain ReAct Agent mit Azure OpenAI

Dieses Notebook zeigt, wie man einen einfachen ReAct Agent mit LangChain und Azure OpenAI baut, der streamt.

In [15]:
# Installation (falls nötig)
# !pip install langchain langchain-openai langchain-community

In [16]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import asyncio

In [17]:
load_dotenv()

True

## 1. Azure OpenAI Setup

In [18]:
# Azure OpenAI Konfiguration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-api-key")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-resource.openai.azure.com/")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

# LLM initialisieren
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    streaming=True,  # Für Streaming-Unterstützung
    callbacks=[StreamingStdOutCallbackHandler()]  # Live-Output
)

In [19]:
llm.invoke("hi")

Hello — how can I help you today?

AIMessage(content='Hello — how can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-5-mini-2025-08-07', 'model_provider': 'openai'}, id='lc_run--019b9282-a345-7f42-bfdd-3f8a8c264ff9', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2. Einfache Tools definieren

In [20]:
@tool
def calculator(expression: str) -> str:
    """Führt einfache mathematische Berechnungen aus.
    
    Args:
        expression: Eine mathematische Expression wie "2 + 3 * 4"
        
    Returns:
        Das Ergebnis der Berechnung als String
    """
    try:
        # Sicherheitscheck - nur erlaubte Operationen
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Fehler: Nur Zahlen und Grundrechenarten (+, -, *, /) erlaubt"
        
        result = eval(expression)
        return f"Ergebnis: {result}"
    except Exception as e:
        return f"Fehler bei der Berechnung: {str(e)}"

@tool
def search_web(query: str) -> str:
    """Simuliert eine Web-Suche (für Demo-Zwecke).
    
    Args:
        query: Suchbegriff
        
    Returns:
        Simuliertes Suchergebnis
    """
    # In der Realität würde hier eine echte Such-API verwendet werden
    mock_results = {
        "wetter berlin": "Das Wetter in Berlin ist sonnig, 22°C",
        "python tutorial": "Python ist eine Programmiersprache. Besuchen Sie python.org für Tutorials",
        "langchain": "LangChain ist ein Framework für LLM-Anwendungen"
    }
    
    for key, result in mock_results.items():
        if key in query.lower():
            return f"Suchergebnis für '{query}': {result}"
    
    return f"Keine Ergebnisse gefunden für: {query}"

# Tools-Liste
tools = [calculator, search_web]

## 3. ReAct Agent erstellen

LangChain bietet vorgefertigte Klassen für ReAct Agents:
- `create_agent`: Erstellt einen ReAct Agent mit vorgefertigtem Prompt
- Agent ist direkt verwendbar (Runnable) - kein AgentExecutor mehr nötig

In [27]:
# ReAct Agent Template (vereinfacht)
react_template = """Du bist ein hilfreicher AI-Assistent. Verwende die folgenden Tools, um Fragen zu beantworten.

Verfügbare Tools:
{tools}

Verwende das folgende Format:
Question: die Eingabe-Frage
Thought: denke Schritt für Schritt
Action: das Tool, das du verwenden möchtest, mit Input
Observation: das Ergebnis des Tools
... (dieses Thought/Action/Observation kann sich wiederholen)
Final Answer: die finale Antwort

Beginne!

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(react_template)

# Agent erstellen
agent = create_agent(llm, tools)

# Agent ist jetzt direkt verwendbar (kein AgentExecutor mehr nötig)
# In LangChain 1.x sind Agents Runnable und können direkt verwendet werden

## 4. Agent testen (ohne Streaming)

In [28]:
# Einfacher Test ohne Streaming
result = agent.invoke({
    "messages": [{"role": "user", "content": "Was ist 15 + 27 mal 3?"}]
})

print("Finale Antwort:")
try:
    # Some runtimes return message objects with attribute access
    print(result['messages'][-1].content)
except Exception:
    # Fallback for dict-like messages
    print(result['messages'][-1].get('content', result['messages'][-1]))

Nach Punkt-vor-Strich: zuerst 27 × 3 = 81, dann 15 + 81 = 96.Finale Antwort:
Nach Punkt-vor-Strich: zuerst 27 × 3 = 81, dann 15 + 81 = 96.


## 5. Agent mit Streaming

Für Streaming verwenden wir die `astream_events` Methode des Agent direkt.

In [30]:
# Streaming-Funktion
async def run_agent_streaming(query: str):
    """Führt den Agent mit Streaming aus"""
    print(f"Frage: {query}")
    print("=" * 50)
    
    async for event in agent.astream_events(
        {"messages": [{"role": "user", "content": query}]},
        version="v1"
    ):
        kind = event["event"]
        if kind == "on_chain_start":
            if event["name"] == "Agent":
                print(f"Starting agent: {event['name']}")
        elif kind == "on_chain_end":
            if event["name"] == "Agent":
                print(f"Done agent: {event['name']}")
        elif kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            print(content, end="", flush=True)
        elif kind == "on_tool_start":
            print(f"\nUsing tool: {event['name']}")
        elif kind == "on_tool_end":
            print(f"Tool result: {event['data']['output']}")

# Beispiel mit Streaming
await run_agent_streaming("Berechne 42 * 7 und suche nach 'python tutorial'")

Frage: Berechne 42 * 7 und suche nach 'python tutorial'

Using tool: search_web
Tool result: content="Suchergebnis für 'python tutorial': Python ist eine Programmiersprache. Besuchen Sie python.org für Tutorials" name='search_web' tool_call_id='call_hKrYmAEATRwuKhdYVSsPafhc'

Using tool: calculator
Tool result: content='Ergebnis: 294' name='calculator' tool_call_id='call_asiNx0CdVyoI0Igdp42hBea4'
DasDas Ergebnis Ergebnis der der Rechnung Rechnung::  4242 × ×  77 = =  294294.

.

SuchSucherergebnisgebnis für für " "pythonpython tutorial tutorial":": Python Python ist ist eine eine Program Programmimierserspracheprache.. Bes Besuchenuchen Sie Sie python python.org.org für für Tutorials Tutorials.

.

SollSoll ich ich Ihnen Ihnen konkrete konkrete Ein Einsteigersteiger‑‑ oder oder Fort Fortgeschgeschrittenrittenenen‑‑RRessessourcenourcen//LinksLinks zusammen zusammenstellenstellen??

## 6. Einfacher Chat-Loop

In [ ]:
async def chat_loop():
    """Einfacher Chat-Loop für Interaktion"""
    print("ReAct Agent Chat (type 'quit' to exit)")
    print("-" * 40)
    
    while True:
        user_input = input("Du: ")
        if user_input.lower() in ['quit', 'exit', 'q']:
            break
            
        await run_agent_streaming(user_input)
        print("\n" + "="*50 + "\n")

# Chat starten (kommentiert aus, da input() in Notebook problematisch ist)
# await chat_loop()

## Zusammenfassung

**Vorgefertigte Klassen in LangChain:**
- `create_agent()`: Erstellt ReAct Agent automatisch
- Agent ist Runnable - direkte Verwendung mit `.invoke()`, `.stream()`, `.astream_events()`
- Streaming über `astream_events()`

**Azure OpenAI Integration:**
- `AzureChatOpenAI` Klasse für Azure Endpoints
- Automatische Streaming-Unterstützung
- Callbacks für Live-Output

**Tools:**
- Einfach mit `@tool` Decorator definierbar
- Automatische Schema-Generierung für LLM